# Atmospheric reaction model useb by Sandu (2001)

In [ ]:
import PyPlot; plt = PyPlot
using LaTeXStrings

using OrdinaryDiffEq

function σ(t)
    # time is given in seconds and must be converted to hours
    TL = mod(t / 3600, 24)
    TR = 4.5  # sunrise at 04:30
    TS = 19.5 # sunset  at 19:30
    ifelse(TR <= TL <= TS, 0.5 + 0.5 * cospi(abs((2TL - TR - TS) / (TS - TR))*((2TL - TR - TS) / (TS - TR))), 0.0)
end

function stratospheric_reaction(du, u, p, t)
    sig = σ(t)
    O1D, O, O3, O2, NO, NO2 = u
    
    k1 = 2.643e-10 * sig^3
    r1 = k1 * O2
    
    k2 = 8.018e-17
    r2 = k2 * O * O2
    
    k3 = 6.120e-04 * sig
    r3 = k3 * O3
    
    k4 = 1.576e-15
    r4 = k4 * O * O3
    
    k5 = 1.070e-03 * sig^2
    r5 = O3
    
    M  = 8.120e16
    k6 = 7.110e-11
    r6 = k6 * M * O1D
    
    k7 = 1.200e-10
    r7 = k7 * O1D * O3
    
    k8 = 6.062e-15
    r8 = k8 * O3 * NO
    
    k9 = 1.069e-11
    r9 = k9 * O * NO2
    
    k10 = 1.289e-2 * sig
    r10 = k10 * NO2
    
    k11 = 1.0e-8
    r11 = k11 * NO * O
    
    # u = O1D, O, O3, O2, NO, NO2
    du[1] = dO1D = -r6 - r7 + r5
    du[2] = dO   = -r2 - r4 - r9 - r11 + 2*r1 + r3 + r6 + r10
    du[3] = dO3  = -r3 - r4 - r5 - r7 - r8 + r2
    du[4] = dO2  = -r1 - r2 + r3 + 2*r4 + r5 + 2*r7 + r8 + r9
    du[5] = dNO  = -r8 - r11 + r9 + r10
    du[6] = dNO2 = -r9 - r10 + r8 + r11
    nothing
end

# the simulation starts at noon and lasts for 72 hours
tspan = (12*3600.0, 84*3600.0)
# u = O1D, O, O3, O2, NO, NO2
u0 = [9.906e01, 6.624e08, 5.326e11, 1.697e16, 4.000e06, 1.093e09]

ode = ODEProblem(stratospheric_reaction, u0, tspan)
sol = solve(ode, Rodas3(), tol=1.0e-12)
# sol = solve(ode, Rodas3(), adaptive=false, dt=0.5*3600)

# plt.plot(sol.t ./ 3600, map(u->u[1], sol.u), label="O1D")
# plt.plot(sol.t ./ 3600, map(u->u[2], sol.u), label="O")
# plt.plot(sol.t ./ 3600, map(u->u[3], sol.u), label="O3")
# plt.plot(sol.t ./ 3600, map(u->u[4], sol.u), label="O2")
# plt.plot(sol.t ./ 3600, map(u->u[5], sol.u), label="NO")
plt.plot(sol.t ./ 3600, map(u->u[6], sol.u), label="NO2")
# plt.plot(sol.t ./ 3600, map(u->u[5]+u[6], sol.u), label="NO + NO2")
# plt.plot(sol.t ./ 3600, map(u->u[1]+u[2]+3u[3]+2u[4]+u[5]+2u[6], sol.u), label="O1D + O + 3 O3 + 2 O2 + NO + 2 NO2")
# plt.yscale("log")
plt.xlabel(L"Time in $h$")
plt.ylabel("Concentrations")
plt.legend();